In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cv2
import os

import skimage.io as ski
from skimage.registration import optical_flow_ilk    # optical flow por Lucas Kanade
from skimage.registration import optical_flow_tvl1   # Optical flow por Total Variation of Linearized Optical Flow
from skimage.color import hsv2rgb
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte

import matplotlib.pyplot as plt
import shutil
import time
import random


#### Define paths

###### Paths

In [78]:
# subdivisão de augmentação

#rotacao = '0 graus'
#rotacao = '90 graus'
#rotacao = '180 graus'
rotacao = '270 graus'

In [79]:
# subdivisão das imagens

letra = 'c'

In [80]:
# diretorio origem das imagens que serão editadas

diretorio = 'Fumaca'
#diretorio = 'Nao_Fumaca'


In [81]:
# diretorio aonde as imagens editadas serão salvas

diretorio_destino = diretorio
#diretorio_destino = 'Fumaca'
#diretorio_destino = 'Nao_Fumaca'


In [82]:
# etapa (treinamento (classificação ou detecção) ou validação (classificação e detecção))

#etapa = 'Treinamento Classificação'
#etapa = 'Treinamento Detecção'
etapa = 'Validacao - Classificação e Deteçcão'

In [83]:
# path aonde as imagens que serão editadas estão localizadas

path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Pre processamento/{etapa}/{diretorio_destino}/3. Augmented/augmented {letra}/{rotacao}'

In [84]:
# path aonde as imagens editadas serão salvas

path2 = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Pre processamento/{etapa}/{diretorio_destino}/4. Motion Flow'


In [85]:
# garante que o diretorio destino (path2) está vazio

assert len(os.listdir(f"{path2}/Dense Optical Flow/motion flow {letra}/{rotacao}")) == 0
assert len(os.listdir(f"{path2}/Optical Flow/motion flow {letra}/{rotacao}")) == 0


In [86]:
f"{path2}/Optical Flow/motion flow {letra}/{rotacao}"

'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Pre processamento/Validacao - Classificação e Deteçcão/Fumaca/4. Motion Flow/Optical Flow/motion flow c/270 graus'

###### Plota algumas imagens

In [ ]:
 # salva os nomes das imagens antes das edições

list_imagens_originais = os.listdir(f'{path}')
list_imagens_originais.sort()

list_imagens_originais[0:5]

[]

In [ ]:
path2

'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Pre processamento/Validacao - Classificação e Deteçcão/Fumaca/4. Motion Flow'

In [ ]:
len(list_imagens_originais)

0

In [ ]:
list_imagens_originais[0:5]

[]

In [ ]:
# salva os nomes das imagens antes das edições

nome_imagens_originais = []

for i in list_imagens_originais:
  nome_imagens_originais.append(i.split('.')[0]) # pega somente o nome da imagem (e exclui a extensão)

In [ ]:
# salva as imagens antes das edições

imagens_originais = [cv2.imread(f'{path}/{i}') for i in list_imagens_originais] #cv2.imread() retorna um array -> compreensão de lista para fazer upload de todas as imagens

In [ ]:
assert type(imagens_originais[0]) == np.ndarray

IndexError: ignored

In [ ]:
i = len(imagens_originais)
i

In [ ]:
coluna = 2
linha = 2# int(i/4) if i%4 == 0 else int((i- i%4)/4)
if linha > 4:
  linha = 4
linha,coluna

In [ ]:
time.sleep(5)

f, coords = plt.subplots(linha,coluna)
for i in range(linha):
  for j in range(coluna):
    coords[i,j].imshow(random.choice(imagens_originais), cmap = 'gray')


#### Optical Flow

##### Funções

In [87]:
# https://github.com/Utkal97/Object-Tracking
# https://github.com/Utkal97/Object-Tracking/blob/main/LucasKanadeOptFlow.py

In [88]:
def inRange( cordinates, limits):
	x,y = cordinates
	X_Limit, Y_Limit = limits
	return 0 <= x and x < X_Limit and 0 <= y and y < Y_Limit


In [89]:
def optical_flow(old_frame, new_frame, window_size, min_quality=0.01):

    max_corners = 10000
    min_distance = 0.1
    feature_list = cv2.goodFeaturesToTrack(old_frame, max_corners, min_quality, min_distance)

    w = int(window_size/2)

    old_frame = old_frame / 255
    new_frame = new_frame / 255

    #Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1, 1], [-1, 1]])
    kernel_y = np.array([[-1, -1], [1, 1]])
    kernel_t = np.array([[1, 1], [1, 1]])

    fx = cv2.filter2D(old_frame, -1, kernel_x)              #Gradient over X
    fy = cv2.filter2D(old_frame, -1, kernel_y)              #Gradient over Y
    ft = cv2.filter2D(new_frame, -1, kernel_t) - cv2.filter2D(old_frame, -1, kernel_t)  #Gradient over Time


    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:        #   for every corner
            j, i = feature.ravel()		#   get cordinates of the corners (i,j). They are stored in the order j, i
            i, j = int(i), int(j)		#   i,j are floats initially

            I_x = fx[i-w:i+w+1, j-w:j+w+1].flatten()
            I_y = fy[i-w:i+w+1, j-w:j+w+1].flatten()
            I_t = ft[i-w:i+w+1, j-w:j+w+1].flatten()

            b = np.reshape(I_t, (I_t.shape[0],1))
            A = np.vstack((I_x, I_y)).T

            U = np.matmul(np.linalg.pinv(A), b)     # Solving for (u,v) i.e., U

            u[i,j] = U[0][0]
            v[i,j] = U[1][0]

    return (u,v)

In [90]:
# antigo, com as setas invertidas

# def drawOnFrame(frame, U, V, output_file):

#     for j in range(frame.shape[0]):
#       for i in range(frame.shape[1]):
#             u, v = U[j][i], V[j][i]

#             if u and v:
#                 frame = cv2.arrowedLine( frame, (i, j), (int(round(i+u)), int(round(j+v))),
#                                         (0, 0, 255),  # red (sistema BGR)
#                                         thickness=1, tipLength = 0.5
#                                     )
#     cv2.imwrite(output_file, frame)


In [91]:
def drawOnFrame(frame, U, V, output_file):

    for j in range(frame.shape[0]):
      for i in range(frame.shape[1]):
            u, v = U[j][i], V[j][i]

            if u and v:

                u*= -1
                v*= -1
                frame = cv2.arrowedLine( frame,(i, j) , (i-int(round(u)), j-int(round(v))),
                                        (0, 0, 255),  # red (sistema BGR)
                                        thickness=1, tipLength = 0.5
                                    )
    cv2.imwrite(output_file, frame)


In [92]:
def drawSeperately(old_frame, new_frame, U, V, output_file):

    displacement = np.ones_like(img2)
    displacement.fill(255.)             #Fill the displacement plot with White background
    line_color =  (0, 0, 0)
    # draw the displacement vectors
    for i in range(img2.shape[0]):
        for j in range(img2.shape[1]):

            start_pixel = (i,j)
            end_pixel = ( int(i+U[i][j]), int(j+V[i][j]) )

            #check if there is displacement for the corner and endpoint is in range
            if U[i][j] and V[i][j] and inRange( end_pixel, img1.shape ):
                displacement = cv2.arrowedLine( displacement, start_pixel, end_pixel, line_color, thickness =2)

    figure, axes = plt.subplots(1,3)
    axes[0].imshow(old_frame, cmap = "gray")
    axes[0].set_title("first image")
    axes[1].imshow(new_frame, cmap = "gray")
    axes[1].set_title("second image")
    axes[2].imshow(displacement, cmap = "gray")
    axes[2].set_title("displacements")
    figure.tight_layout()
    plt.savefig(output_file, bbox_inches = "tight", dpi = 200)



##### Gera imagens

In [93]:
img = os.listdir(f'{path}')
img[:5]

['270 graus201.png',
 '270 graus210.png',
 '270 graus202.png',
 '270 graus209.png',
 '270 graus211.png']

In [94]:
for i in img:
  i = i.split('graus')[1]
  i = i.split('.')[0]
  print(i)

201
210
202
209
211
208
204
207
206
203
205
212
214
213
215
218
216
217


In [95]:
# coleta a numeração da imagem, tira o resto do nome e ordena sequencialmente as imagens (list.sort() nao garante ordenação correta)

imagens = []

for i in img:
  i = i.split('.')[0]

  if 'Graus' in i or 'graus' in i:
    i = i.split('graus')[1]
    i = i.split('.')[0]

  imagens.append(int(i))

imagens.sort()
imagens[:5]


[201, 202, 203, 204, 205]

In [96]:
for i in range(0,len(imagens)-1):
  print(imagens[i], imagens[i+1])

201 202
202 203
203 204
204 205
205 206
206 207
207 208
208 209
209 210
210 211
211 212
212 213
213 214
214 215
215 216
216 217
217 218


In [97]:
# garante que o diretorio destino (path2) está vazio

assert len(os.listdir(f"{path2}/Optical Flow/motion flow {letra}/{rotacao}")) == 0


In [98]:
ws = 15  # vizinhança para considerar ao calcular o motion flow

In [99]:
for i in range(0, len(imagens)-1):
  # lê a imagem anterior
  img1 = cv2.imread(f'{path}/{rotacao}{imagens[i]}.png')
  img1_cinza = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

  # lê a imagem atual
  img2 = cv2.imread(f'{path}/{rotacao}{imagens[i+1]}.png')
  img2_cinza = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

  # faz uma cópia da imagem atual (para ser plotado o vetor u,v de movimentação )
  im_save = np.copy(img2)

  # calcula o optical flow (Lucas Kanade) entre as duas imagens
  #flow = optical_flow_ilk(img1_cinza,img2_cinza)
  #u,v = flow[0,:,:],flow[1,:,:]

  u,v = optical_flow(img1_cinza, img2_cinza, window_size = ws)

  # plota os vetores u,v calculados na imagem e salva a imagem gerada no diretorio de motion flow
  drawOnFrame(im_save ,u, v, f"{path2}/Optical Flow/motion flow {letra}/{rotacao}/{rotacao} {imagens[i+1]}(mf {ws}x{ws}).png")



#### Dense Optica Flow

In [ ]:
#https://gist.github.com/insaneyilin/e9120bdcde690ee52c40fa4bdf26ae19

In [ ]:
img = os.listdir(f'{path}')

In [ ]:
# coleta a numeração da imagem, tira o resto do nome e ordena sequencialmente as imagens (list.sort() nao garante ordenação correta)

imagens = []

for i in img:
  i = i.split('.')[0]


  imagens.append(int(i))

imagens.sort()
imagens[:5]


[106, 107, 108, 109, 110]

In [ ]:
for i in range(0,len(imagens)-1):
  print(imagens[i], imagens[i+1])

106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 117
117 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135


In [ ]:
# garante que o diretorio destino (path2) está vazio

if etapa == 'Treinamento Classificação' or  etapa == 'Treinamento Detecção':
  assert len(os.listdir(f"{path2}/Dense Optical Flow/motion flow {letra}/{rotacao}")) == 0


elif etapa == 'Validacao - Classificação e Deteçcão':
  assert len(os.listdir(f"{path2}/Dense Optical Flow/{rotacao}")) == 0


In [ ]:
for i in range(0, len(imagens)-1):
  # lê a imagem anterior
  img1 = ski.imread(f'{path}/{imagens[i]}.png')
  img1_cinza = rgb2gray(img1)

  # lê a imagem atual
  img2 = ski.imread(f'{path}/{imagens[i+1]}.png')
  img2_cinza = rgb2gray(img2)

  # calcula fluxo optico
  flow = optical_flow_tvl1(img1_cinza,img2_cinza)

  # calcula fluxo optico denso
  flow_x = flow[1, :, :]
  flow_y = flow[0, :, :]
  mag_map = np.hypot(flow_x, flow_y)  # magnitude
  mag_map *= 255 / mag_map.max()
  mag_map = mag_map.astype('uint8')

  angle_map = np.arctan2(flow_y, flow_x) + np.pi  # direção
  angle_map *= (180./np.pi)
  angle_map = np.clip(angle_map, 0, 180)
  angle_map = angle_map.astype('uint8')

  # converte para espaco de cores hsv
  s_map = (np.ones(mag_map.shape) * 255).astype('uint8')
  img_hsv = np.dstack((angle_map, s_map, mag_map))

  # converte para espaco de cores rgb
  img_rgb = hsv2rgb(img_hsv)
  img_rgb = img_as_ubyte(img_rgb)

  # salva as imagens
  ski.imsave(f"{path2}/Dense Optical Flow/motion flow {letra}/{rotacao}/DenseOF_{imagens[i+1]}({rotacao}).png",img_rgb)


#### Renomeia as imagens para o diretorio (nao usar)

In [ ]:
rotacao = ['0 graus', '90 graus', '180 graus','270 graus']

In [ ]:
letra = ['a','b','c','d','e','f']

In [ ]:
flow = 'Optical Flow'
#flow = 'Dense Optical Flow'

In [ ]:
diretorio = ['Fumaca', 'Nao_Fumaca']

In [ ]:
diretorio_destino = diretorio


In [ ]:
treinamento = 'Treinamento Classificação'
#treinamento = 'Treinamento Detecção'

In [ ]:
# path aonde as imagens editadas estão localizadas

path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Imagens/{treinamento}/{diretorio_destino}/4. Motion Flow/{rotacao}'

In [ ]:
for d in diretorio:
  for l in letra:
    for r in rotacao:
      path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Pre processamento/{treinamento}/{d}/4. Motion Flow/{flow}/motion flow {l}/{r}'
      for img in os.listdir(path):
        os.rename(f'{path}/{img}',f'{path}/{r}{img}')


#### Move as imagens para o diretorio (nao usar)

##### Paths

In [ ]:
rotacao = ['0 graus', '90 graus', '180 graus','270 graus']

In [ ]:
letra = ['a','b','c','d','e','f']

In [ ]:
flow = 'Optical Flow'
#flow = 'Dense Optical Flow'

In [ ]:
diretorio = ['Fumaca', 'Nao_Fumaca']

In [ ]:
treinamento = 'Treinamento Classificação'
#treinamento = 'Treinamento Detecção'

In [ ]:
# path aonde as imagens editadas estão localizadas

path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Imagens/{treinamento}/{diretorio}/4. Motion Flow/{rotacao}'

In [ ]:
# para mover imagens
for d in diretorio:
  for l in letra:
    for r in rotacao:
      path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Pre processamento/{treinamento}/{d}/4. Motion Flow/{flow}/motion flow {l}/{r}'
      path2 = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Treinamento/{treinamento}/{d}'
      for img in os.listdir(path):
        shutil.copy(f'{path}/{img}',f'{path2}/{img}')

### lk direto
  

##### Funções

In [ ]:
# https://github.com/Utkal97/Object-Tracking
# https://github.com/Utkal97/Object-Tracking/blob/main/LucasKanadeOptFlow.py

In [ ]:
def inRange( cordinates, limits):
	x,y = cordinates
	X_Limit, Y_Limit = limits
	return 0 <= x and x < X_Limit and 0 <= y and y < Y_Limit


In [ ]:
def optical_flow(old_frame, new_frame, window_size, min_quality=0.01):

    max_corners = 10000
    min_distance = 0.1
    feature_list = cv2.goodFeaturesToTrack(old_frame, max_corners, min_quality, min_distance)

    w = int(window_size/2)

    old_frame = old_frame / 255
    new_frame = new_frame / 255

    #Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1, 1], [-1, 1]])
    kernel_y = np.array([[-1, -1], [1, 1]])
    kernel_t = np.array([[1, 1], [1, 1]])

    fx = cv2.filter2D(old_frame, -1, kernel_x)              #Gradient over X
    fy = cv2.filter2D(old_frame, -1, kernel_y)              #Gradient over Y
    ft = cv2.filter2D(new_frame, -1, kernel_t) - cv2.filter2D(old_frame, -1, kernel_t)  #Gradient over Time


    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:        #   for every corner
            j, i = feature.ravel()		#   get cordinates of the corners (i,j). They are stored in the order j, i
            i, j = int(i), int(j)		#   i,j are floats initially

            I_x = fx[i-w:i+w+1, j-w:j+w+1].flatten()
            I_y = fy[i-w:i+w+1, j-w:j+w+1].flatten()
            I_t = ft[i-w:i+w+1, j-w:j+w+1].flatten()

            b = np.reshape(I_t, (I_t.shape[0],1))
            A = np.vstack((I_x, I_y)).T

            U = np.matmul(np.linalg.pinv(A), b)     # Solving for (u,v) i.e., U

            u[i,j] = U[0][0]
            v[i,j] = U[1][0]

    return (u,v)

In [ ]:
def drawOnFrame(frame, U, V, output_file):

    for j in range(frame.shape[0]):
      for i in range(frame.shape[1]):
            u, v = U[j][i], V[j][i]

            if u and v:
                frame = cv2.arrowedLine( frame, (i, j), (int(round(i+u)), int(round(j+v))),
                                        (0, 0, 255),  # red (sistema BGR)
                                        thickness=1, tipLength = 0.5
                                    )
    cv2.imwrite(output_file, frame)


In [ ]:
def drawSeperately(old_frame, new_frame, U, V, output_file):

    displacement = np.ones_like(img2)
    displacement.fill(255.)             #Fill the displacement plot with White background
    line_color =  (0, 0, 0)
    # draw the displacement vectors
    for i in range(img2.shape[0]):
        for j in range(img2.shape[1]):

            start_pixel = (i,j)
            end_pixel = ( int(i+U[i][j]), int(j+V[i][j]) )

            #check if there is displacement for the corner and endpoint is in range
            if U[i][j] and V[i][j] and inRange( end_pixel, img1.shape ):
                displacement = cv2.arrowedLine( displacement, start_pixel, end_pixel, line_color, thickness =2)

    figure, axes = plt.subplots(1,3)
    axes[0].imshow(old_frame, cmap = "gray")
    axes[0].set_title("first image")
    axes[1].imshow(new_frame, cmap = "gray")
    axes[1].set_title("second image")
    axes[2].imshow(displacement, cmap = "gray")
    axes[2].set_title("displacements")
    figure.tight_layout()
    plt.savefig(output_file, bbox_inches = "tight", dpi = 200)



#### gera

In [ ]:
ws = 15  # vizinhança para considerar ao calcular o motion flow

In [ ]:
rotacoes = ['0 graus','90 graus','180 graus','270 graus']
letras = 'e'#['a','b','c','d','e','f']
diretorios = ['Nao_Fumaca']#,'Fumaca']

In [ ]:
for d in diretorios:
  diretorio = d

  for l in letras:
    letra = l

    for r in rotacoes:
      rotacao = r

      path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Imagens/{diretorio}/3. Augmented/augmented {letra}/{rotacao}'
      path2 = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Imagens/{diretorio_destino}/4. Motion Flow/'


      img = os.listdir(f'{path}')

      imagens = []

      for i in img:
        i = i.split('.')[0]

        if 'Graus' in i:
          i = i.split(')')[1]

        imagens.append(int(i))

      imagens.sort()


      for i in range(0, len(imagens)-1):
        # lê a imagem anterior
        img1 = cv2.imread(f'{path}/{imagens[i]}.png')
        img1_cinza = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

        # lê a imagem atual
        img2 = cv2.imread(f'{path}/{imagens[i+1]}.png')
        img2_cinza = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

        # faz uma cópia da imagem atual (para ser plotado o vetor u,v de movimentação )
        im_save = np.copy(img2)

        u,v = optical_flow(img1_cinza, img2_cinza, window_size = ws)

        # plota os vetores u,v calculados na imagem e salva a imagem gerada no diretorio de motion flow
        drawOnFrame(im_save ,u, v, f"{path2}/Optical Flow/motion flow {letra}/{rotacao}/{imagens[i+1]}(mf {ws}x{ws}).png")




In [ ]:
for d in diretorios:
  diretorio = d

  for l in letras:
    letra = l

    for r in rotacoes:
      rotacao = r

      path = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Imagens/{diretorio}/3. Augmented/augmented {letra}/{rotacao}'
      path2 = f'/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Imagens/{diretorio_destino}/4. Motion Flow/'


      imagens = []

      for i in img:
        i = i.split('.')[0]

        if 'Graus' in i:
          i = i.split(')')[1]

        imagens.append(int(i))

      imagens.sort()

      print(imagens)

In [ ]:
type(u),u.shape